In [ ]:
#1. 필요한 모듈

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime
import re

In [ ]:
# 2. 기사 제목 url 크롤링 함수 선언
def get_company_url(a):
    driver.get(a)
    # driver를 이용하여 가져오기
    contents_url = driver.find_elements_by_css_selector("a.l.lLrAF")
    contents_comp = driver.find_elements_by_css_selector('div.gG0TJc')


    # 각 url과 뉴스사를 리스트로 만들기
    list_company = []
    for i in contents_comp:
        a = i.find_elements_by_css_selector('span.xQ82C.e8fRJf')
        for j in a:
            list_company.append(j.text)
        
    list_url = []
    for i in contents_url:
        list_url.append(i.get_attribute('href'))
    
    list_title = []
    for i in contents_url:
        list_title.append(i.text)

    # dataframe으로 만들기
    list_merge = [list_company, list_title, list_url]
    col_names = ['company', 'title', 'url']
    df_ = pd.DataFrame(list_merge)
    df = df_.T
    df.columns = col_names
    
    return df
    # company filtering 함수로 보내기
    # filter_company(df)

def filter_company(b):
    
    # company에 있는 뉴스사들만 추출하기
    company_list = ['newsBTC', 'Cointelegraph', 'Coindesk', 'BelnCrypto', 'Bitcoinist', 'Bitcoin News',
                    'CNBC', 'The Guardian', 'BBC news', 'Reuters', 'Business Week', 'The Economist',
                    'Bloomberg', 'The Wall Street Journal', 'Forbes']
    filter_df = b[b['company'].isin(company_list)]
    
    return filter_df
    # date_column(filter_df)    
    
def date_column(c):
    date_list = []
    for i in range(len(c.index)):
        date_list.append(DATE)
    c.insert(0, 'date', date_list)
    return c

In [ ]:
#3. 특정 날짜에 대한 비트코인 검색 후 첫페이지 url

date_30 = [4, 6, 9, 11]
date_28 = [2]
date_31 = [1, 3, 5, 7, 8, 10, 12]

def datetime(y,m1,m2):  # 2017,9,12 하면 2017년 9월에서 12월까지의 일별 url
    url_result=dict() #결과

    for i in range(m1,m2+1):
        month=i
        if i in date_31:
            for j in range(1,32):
                day=j
                month_day=str(y)+"-"+str(i)+"-"+str(j)
                date = [str(y),str(i),str(j)]
                duration = '%2Ccd_min%3A{0}%2F{1}%2F{2}%2Ccd_max%3A{3}%2F{4}%2F{5}'.format(date[1], date[2], date[0], date[1], date[2], date[0])
                base_url = 'https://www.google.com/search?q=bitcoin&biw=1005&bih=842&source=lnt&tbs=cdr%3A1'+ duration + '&tbm=nws'
                url_result[month_day]=base_url



        elif i in date_28:
            for j in range(1,29):
                day=j
                month_day=str(y)+"-"+str(i)+"-"+str(j)
                date = [str(y),str(i),str(j)]
                duration = '%2Ccd_min%3A{0}%2F{1}%2F{2}%2Ccd_max%3A{3}%2F{4}%2F{5}'.format(date[1], date[2], date[0], date[1], date[2], date[0])
                base_url = 'https://www.google.com/search?q=bitcoin&biw=1005&bih=842&source=lnt&tbs=cdr%3A1'+ duration + '&tbm=nws' 
                url_result[month_day]=base_url
        else:
            for j in range(1,31):
                day=j
                month_day=str(y)+"-"+str(i)+"-"+str(j)
                date = [str(y),str(i),str(j)]
                duration = '%2Ccd_min%3A{0}%2F{1}%2F{2}%2Ccd_max%3A{3}%2F{4}%2F{5}'.format(date[1], date[2], date[0], date[1], date[2], date[0])
                base_url = 'https://www.google.com/search?q=bitcoin&biw=1005&bih=842&source=lnt&tbs=cdr%3A1'+ duration + '&tbm=nws'
                url_result[month_day]=base_url
    return url_result




d1= datetime(2017,9,11)
d2= datetime(2018,1,12)
d3= datetime(2019,1,9)
d1.update(d2)
d1.update(d3) 

DATE= [a for a in d1.keys()]
url_list=[a for a in d1.values()]

In [ ]:
#4. 페이지 넘기며 신문사, 뉴스, 날짜 수집

driver = webdriver.Chrome(r"C:\Users\rok80\Anaconda3\네이버뉴스\chromedriver_win32\chromedriver.exe")
driver.set_page_load_timeout("10")


FINAL=[]
for DATE, URL in zip(DATE[:5], url_list[:5]): 
    
    time.sleep(1)
    driver.get(URL) # 페이지 입장
    time.sleep(3)
    
    print("웹페이지 인덱스" , url_list.index(URL), "/",len(url_list) )
    r1 = get_company_url(URL)
    r2 = filter_company(r1)
    r3 = date_column(r2)
    FINAL.append(r3)

    for count in range(999):
        if count==0:
            try:
                soup = BeautifulSoup(driver.page_source)
                h1= soup.find_all("a", class_="pn")
                h2='http://www.google.com'+h1[0]['href'] # 다음버튼 url
                driver.get(h2)
                time.sleep(3)
            except Exception as ex:
                print('더이상 "페이지" 없음', ex)
                break
        else:
            try:
                soup = BeautifulSoup(driver.page_source)
                h1= soup.find_all("a", class_="pn")
                h2='http://www.google.com'+h1[1]['href'] # 다음버튼 url
                driver.get(h2)
                time.sleep(3)
            except Exception as ex:
                print('더이상 "페이지" 없음', ex)
                break
    
driver.close()    

웹페이지 인덱스 0 / 729
더이상 "페이지" 없음 list index out of range
웹페이지 인덱스 1 / 729
더이상 "페이지" 없음 list index out of range
웹페이지 인덱스 2 / 729
더이상 "페이지" 없음 list index out of range
웹페이지 인덱스 3 / 729
더이상 "페이지" 없음 list index out of range
웹페이지 인덱스 4 / 729
더이상 "페이지" 없음 list index out of range


In [ ]:
pd.concat(FINAL)

,date,company,title,url
0,2017-9-1,Forbes,Why Bitcoin Prices Have Risen More Than 400% T...,https://www.forbes.com/sites/cbovaird/2017/09/...
7,2017-9-1,CNBC,Here's why investors have to pay a premium to ...,https://www.cnbc.com/video/2017/09/01/heres-wh...
0,2017-9-3,Forbes,Where Does Bitcoin Fit In Your Portfolio?,https://www.forbes.com/sites/petertchir/2017/0...
1,2017-9-3,Bloomberg,"Sequoia, IDG to Invest in China Bitcoin Mining...",https://www.bloomberg.com/news/articles/2017-0...
5,2017-9-3,Forbes,Russia Opens First Criminal Case Involving Bit...,https://www.forbes.com/sites/kenrapoza/2017/09...
0,2017-9-4,CNBC,Bitcoin price drops $200 after new ruling from...,https://www.cnbc.com/2017/09/04/bitcoin-digita...
4,2017-9-4,The Guardian,London developer to allow rental tenants to pa...,https://www.theguardian.com/money/2017/sep/04/...
1,2017-9-5,CNBC,Real estate project in Dubai to be the 'first ...,https://www.cnbc.com/2017/09/05/dubai-real-est...
6,2017-9-5,Forbes,12 Reasons Gold Is Better Than Bitcoin,https://www.forbes.com/sites/simonconstable/20...


In [ ]:
import pickle
with open('bitcoinDF_19.9-10.pkl', 'rb') as f:
    bitcoinDF = pickle.load(f)

In [ ]:
bitcoinDF['company'].value_counts()

Cointelegraph                   150
Bitcoin News (press release)     60
newsBTC                          47
Forbes                           46
Bitcoinist                       45
                               ... 
The Economist                     1
Wall Street Journal (blog)        1
Business Insider Nordic           1
SC Magazine UK                    1
Crain's Chicago Business          1
Name: company, Length: 68, dtype: int64